# Feature Selection


{{ badge }}

Feature selection is a process where you automatically or manually select those features in your data that contribute most to the prediction variable or output in which you are interested.

There are many ways to perform feature selection. Some methods include:

- Using a correlation matrix to select features that are highly correlated with the output variable
- Using a statistical tests (e.g., $x^2$) to measure the statistical significance of each feature in relation to the label. 
- Using a recursive feature elimination algorithm to automatically select features that are most relevant to the output variable

Feature selection is important in machine learning because it can help you reduce the amount of data you need to work with, which in turn can reduce the amount of time and resources required to train and tune your machine learning model. Additionally, by selecting only the most relevant features, you can improve the interpretability of your machine learning model and make overfitting less likley to occure.

There are many feature selection techniques supported by Scikit Learn. We will go through some of them and if you wish to learn more visit [Scikit Learn's Documentation](https://scikit-learn.org/stable/modules/feature_selection.html).

In [7]:
from sklearn import datasets, model_selection, feature_selection, svm, metrics

In [8]:
# Load iris dataset
x, y = datasets.load_breast_cancer(return_X_y=True)

x.shape

(569, 30)

## Feature Selection Methods

### Univariate Feature Selection

Univariate feature selection works by selecting the best features based on univariate statistical tests. It can be seen as a preprocessing step to an estimator.

We will be using the `SelectKBest` class from Scikit Learn which returing the best $k$ features using some scoring method based on a statistical significance test like $x^2$ (chi-square).

Scikit Learn includes additional scoring functions like:

- For classification: `chi2`, `f_classif`, `mutual_info_classif`
- For regression: `f_regression`, `mutual_info_regression`



In [9]:
# Define the feature selector by selecing the scoring function and the number of features to select
feature_selector = feature_selection.SelectKBest(feature_selection.chi2, k=10)

# Use fit transform to train the feature selector and return the best 10 features
x_new = feature_selector.fit_transform(x, y)

# New X will have 10 features instead of 30
x_new.shape

(569, 10)

### Recursive Feature Elimination

Given an external estimator that assigns weights to features (e.g., the coefficients/weights of a linear model), the goal of recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through any specific attribute (such as `coef_`, `feature_importances_`). Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.



In [10]:
# We use a linear kernel because RFE requires a model that has a coef_ or feature_importances_ attributes
svc = svm.SVC(kernel='linear')

# Define the feature selector by selecing the estimator and the number of features to select, and the number of features to remove in each iteration
rfe = feature_selection.RFE(estimator=svc, n_features_to_select=10, step=1)

# Use fit transform to train the feature selector and return the best 10 features
x_new = rfe.fit_transform(x, y)

# New X will have 10 features instead of 30
x_new.shape

(569, 10)

## Training & Evaluating

In [11]:
use_pruned_features = True #@param {type:"boolean"}
x_final = x_new if use_pruned_features else x

x_train, x_test, y_train, y_test = model_selection.train_test_split(x_final, y, test_size=0.3, stratify=y, random_state=42)

In [12]:
print(f"Training model on {x_train.shape[1]} features")

# Define a Support Vector Machine classifier with default configuration
model = svm.SVC()

# Train model using the training dataset
model.fit(x_train, y_train)

Training model on 10 features


SVC()

In [13]:
# Use the model to predict the testing set to prepare for calculating the metrics
y_pred = model.predict(x_test)

# Calculate and print relevant scores
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.9005847953216374
Precision: 0.8813559322033898
Recall: 0.9719626168224299


# Feature selection using Select From Model

[SelectFromModel](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel) is a meta-transformer that can be used alongside any estimator that assigns importance to each feature through a specific attribute (such as coef_, feature_importances_) or via an importance_getter callable after fitting. 

The features are considered unimportant and removed if the corresponding importance of the feature values are below the provided threshold parameter. 

Apart from specifying the threshold numerically, there are built-in heuristics for finding a threshold using a string argument. Available heuristics are “mean”, “median” and float multiples of these like “0.1*mean”. 

In combination with the threshold criteria, one can use the max_features parameter to set a limit on the number of features to select.

## Define and Train the Linear Model

In [28]:
lsvc = svm.LinearSVC()
lsvc.fit(x, y)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC()

## Use the Trained Linear Model to Get the Best Features

In [29]:
model = feature_selection.SelectFromModel(lsvc, prefit=True)
x_new =model.transform(x)# get the best features using the pretrained model
x_new.shape

(569, 9)

## Split the Datase to Train and Test 

In [30]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_new, y, test_size=0.3, stratify=y, random_state=42)

## Training & Evaluating

In [31]:
print(f"Training model on {x_train.shape[1]} features")

# Define a Support Vector Machine classifier with default configuration
model = svm.SVC()

# Train model using the training dataset
model.fit(x_train, y_train)

Training model on 9 features


SVC()

In [32]:
# Use the model to predict the testing set to prepare for calculating the metrics
y_pred = model.predict(x_test)

# Calculate and print relevant scores
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.9415204678362573
Precision: 0.9292035398230089
Recall: 0.9813084112149533
